<a href="https://colab.research.google.com/github/mtkrahling/-kaggle_projects/blob/main/Titanic_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [ ]:
train_data = pd.read_csv("train_data.csv")

0      25.140620
1      38.233441
2      25.140620
3      38.233441
4      25.140620
         ...    
886    29.877630
887    38.233441
888    25.140620
889    38.233441
890    25.140620
Name: Age, Length: 891, dtype: float64

In [ ]:
features = ["Pclass", "Sex","Fare","Age", "Embarked", "SibSp"]

X = pd.get_dummies(train_data[features], dtype=int)
y = train_data["Survived"]
group_means = train_data.groupby("Pclass")["Age"].transform('mean')
fare_mean=train_data["Fare"].mean()
X['Age'].fillna(value=group_means, inplace=True)
X.loc[X["Fare"] <= 7.775, "Fare"] = 0
X.loc[(X["Fare"] > 7.775) & (X["Fare"] <= 8.662), "Fare"] = 1
X.loc[(X["Fare"] > 8.662) & (X["Fare"] <= 14.454), "Fare"] = 2
X.loc[(X["Fare"] > 14.454) & (X["Fare"] <= 26), "Fare"] = 3
X.loc[(X["Fare"] > 26) & (X["Fare"] <= 52.369), "Fare"] = 4
X.loc[(X["Fare"] > 52.369), "Fare"] = 5


0      22.00000
1      38.00000
2      26.00000
3      35.00000
4      35.00000
         ...   
886    27.00000
887    19.00000
888    25.14062
889    26.00000
890    32.00000
Name: Age, Length: 891, dtype: float64

In [ ]:
clf2 = RandomForestClassifier(n_estimators=50, max_depth=6, random_state=13)
clf = xgb.XGBClassifier(n_estimators=100, max_depth=5, random_state=1)
clf.fit(X,y)
clf2.fit(X,y)

test_data = pd.read_csv("test_data.csv")
X_test = test_data[features]
X_test = pd.get_dummies(X_test[features], dtype=int)
X_test['Age'].fillna(value=group_means, inplace=True)
X_test['Fare'].fillna(value=fare_mean, inplace=True)
X_test.loc[X_test["Fare"] <= 7.775, "Fare"] = 0
X_test.loc[(X_test["Fare"] > 7.775) & (X_test["Fare"] <= 8.662), "Fare"] = 1
X_test.loc[(X_test["Fare"] > 8.662) & (X_test["Fare"] <= 14.454), "Fare"] = 2
X_test.loc[(X_test["Fare"] > 14.454) & (X_test["Fare"] <= 26), "Fare"] = 3
X_test.loc[(X_test["Fare"] > 26) & (X_test["Fare"] <= 52.369), "Fare"] = 4
X_test.loc[(X_test["Fare"] > 52.369), "Fare"] = 5
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      418 non-null    int64  
 1   Fare        418 non-null    float64
 2   Age         418 non-null    float64
 3   SibSp       418 non-null    int64  
 4   Sex_female  418 non-null    int64  
 5   Sex_male    418 non-null    int64  
 6   Embarked_C  418 non-null    int64  
 7   Embarked_Q  418 non-null    int64  
 8   Embarked_S  418 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 29.5 KB


In [ ]:
y_preds = clf2.predict(X_test)

y_preds_series = pd.Series(y_preds, index=test_data.index, name="Survived")
vegso_df = pd.concat([test_data,y_preds_series], axis=1)
vegso_df = vegso_df[["PassengerId", "Survived"]]
vegso_df
vegso_df.to_csv("/content/drive/MyDrive/final10.csv", index=False)